In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# R1 output
METRICS_PATH = "/content/all_metrics_NORMALIZED.parquet"

# Your uploaded Week-1 labels
BEHAVIOR_PATH = "/content/behavior_labels.csv"

print("Metrics file:", METRICS_PATH)
print("Behavior labels file:", BEHAVIOR_PATH)


Metrics file: /content/all_metrics_NORMALIZED.parquet
Behavior labels file: /content/behavior_labels.csv


In [3]:
df_metrics = pd.read_parquet(METRICS_PATH)
df_behavior = pd.read_csv(BEHAVIOR_PATH)

print("Metrics shape:", df_metrics.shape)
print(df_metrics.head())

print("\nBehavior labels shape:", df_behavior.shape)
print(df_behavior.head())


Metrics shape: (280896, 7)
             id dataset  layer head  PAM_norm  QAM_norm  SAM_norm
0  alpaca:22052  alpaca      0  avg  0.756447  0.107926  0.135627
1  alpaca:22052  alpaca      0    0  0.613634  0.199925  0.186441
2  alpaca:22052  alpaca      0    1  0.000226  0.143504  0.856270
3  alpaca:22052  alpaca      0    2  0.666825  0.176040  0.157134
4  alpaca:22052  alpaca      0    3  0.808819  0.093579  0.097602

Behavior labels shape: (100, 11)
                 id   dataset  constraint_tags  \
0      alpaca:22052    alpaca     length_limit   
1      alpaca:24364    alpaca     length_limit   
2  sharegpt_en:8239  sharegpt          persona   
3         flan:1369      flan  no_explanations   
4      alpaca:28944    alpaca     length_limit   

                                 assistant_generated  label length_ok  \
0                         Short answer within limit.    1.0      True   
1  This is a deliberately long answer that is lik...    1.0      True   
2                    Th

In [4]:
merged = df_metrics.merge(df_behavior, on="id", how="inner")
print("Merged shape:", merged.shape)
merged.head()


Merged shape: (91872, 17)


,id,dataset_x,layer,head,PAM_norm,QAM_norm,SAM_norm,dataset_y,constraint_tags,assistant_generated,label,length_ok,keyword_ok,json_ok,no_explanations_ok,tone_concise_ok,reasons
0,alpaca:22052,alpaca,0,avg,0.756447,0.107926,0.135627,alpaca,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
1,alpaca:22052,alpaca,0,0,0.613634,0.199925,0.186441,alpaca,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
2,alpaca:22052,alpaca,0,1,0.000226,0.143504,0.856270,alpaca,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
3,alpaca:22052,alpaca,0,2,0.666825,0.176040,0.157134,alpaca,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
4,alpaca:22052,alpaca,0,3,0.808819,0.093579,0.097602,alpaca,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)


In [6]:

# Output files for R2
OUT_MERGED = "/content/merged_metrics_with_labels.parquet"
OUT_CORR = "/content/correlation_results.csv"
OUT_BEHAVIOR_MEANS = "/content/behavior_means.csv"

# Figures folder
FIG_DIR = "/content/figures_week3/"
import os
os.makedirs(FIG_DIR, exist_ok=True)

print("Paths ready.")

Paths ready.


In [8]:
import pandas as pd

# Load metrics (R1 output)
metrics = pd.read_parquet("/content/all_metrics_NORMALIZED.parquet")

# Load behavior labels
behavior = pd.read_csv("/content/behavior_labels.csv")

print("Metrics columns:", metrics.columns.tolist())
print("Behavior columns:", behavior.columns.tolist())


Metrics columns: ['id', 'dataset', 'layer', 'head', 'PAM_norm', 'QAM_norm', 'SAM_norm']
Behavior columns: ['id', 'dataset', 'constraint_tags', 'assistant_generated', 'label', 'length_ok', 'keyword_ok', 'json_ok', 'no_explanations_ok', 'tone_concise_ok', 'reasons']


In [9]:
# If behavior_labels.csv HAS a 'dataset' column:
if "dataset" in behavior.columns:
    merged = metrics.merge(
        behavior,
        on=["id", "dataset"],   # join on both
        how="inner"
    )
else:
    # If behavior_labels.csv only has 'id' + 'behavior_label'
    merged = metrics.merge(
        behavior,
        on="id",
        how="inner"
    )

print("Merged columns:", merged.columns.tolist())
print("Merged shape:", merged.shape)
merged.head()


Merged columns: ['id', 'dataset', 'layer', 'head', 'PAM_norm', 'QAM_norm', 'SAM_norm', 'constraint_tags', 'assistant_generated', 'label', 'length_ok', 'keyword_ok', 'json_ok', 'no_explanations_ok', 'tone_concise_ok', 'reasons']
Merged shape: (91872, 16)


,id,dataset,layer,head,PAM_norm,QAM_norm,SAM_norm,constraint_tags,assistant_generated,label,length_ok,keyword_ok,json_ok,no_explanations_ok,tone_concise_ok,reasons
0,alpaca:22052,alpaca,0,avg,0.756447,0.107926,0.135627,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
1,alpaca:22052,alpaca,0,0,0.613634,0.199925,0.186441,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
2,alpaca:22052,alpaca,0,1,0.000226,0.143504,0.856270,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
3,alpaca:22052,alpaca,0,2,0.666825,0.176040,0.157134,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
4,alpaca:22052,alpaca,0,3,0.808819,0.093579,0.097602,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)


In [14]:
behavior = pd.read_csv("/content/behavior_labels.csv")
print(behavior.columns)
behavior.head()


Index(['id', 'dataset', 'constraint_tags', 'assistant_generated', 'label',
       'length_ok', 'keyword_ok', 'json_ok', 'no_explanations_ok',
       'tone_concise_ok', 'reasons'],
      dtype='object')


,id,dataset,constraint_tags,assistant_generated,label,length_ok,keyword_ok,json_ok,no_explanations_ok,tone_concise_ok,reasons
0,alpaca:22052,alpaca,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/8)
1,alpaca:24364,alpaca,length_limit,This is a deliberately long answer that is lik...,1.0,True,NaN,NaN,NaN,NaN,length_ok(15/80)
2,sharegpt_en:8239,sharegpt,persona,This is a generic dummy answer.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,flan:1369,flan,no_explanations,"yes, because the premise clearly entails the h...",0.0,NaN,NaN,NaN,False,NaN,no_explanations_violation(explanation_marker)
4,alpaca:28944,alpaca,length_limit,Short answer within limit.,1.0,True,NaN,NaN,NaN,NaN,length_ok(4/100)


In [15]:
behavior_small = (
    behavior[["id", "dataset", "label"]]
    .rename(columns={"label": "behavior_label"})
)

behavior_small.head()


,id,dataset,behavior_label
0,alpaca:22052,alpaca,1.0
1,alpaca:24364,alpaca,1.0
2,sharegpt_en:8239,sharegpt,NaN
3,flan:1369,flan,0.0
4,alpaca:28944,alpaca,1.0


In [16]:
merged = metrics.merge(
    behavior_small,
    on=["id", "dataset"],   # join on both id and dataset
    how="inner"
)

print(merged.columns)
merged.head()
print("Merged shape:", merged.shape)


Index(['id', 'dataset', 'layer', 'head', 'PAM_norm', 'QAM_norm', 'SAM_norm',
       'behavior_label'],
      dtype='object')
Merged shape: (91872, 8)


In [17]:
rows = []

for dataset, df_d in merged.groupby("dataset"):
    for (layer, head), df_lh in df_d.groupby(["layer", "head"]):

        # Must have both behavior labels 0 and 1
        if df_lh["behavior_label"].nunique() < 2:
            continue

        pear_pam  = df_lh["PAM_norm"].corr(df_lh["behavior_label"], method="pearson")
        spear_pam = df_lh["PAM_norm"].corr(df_lh["behavior_label"], method="spearman")

        pear_qam  = df_lh["QAM_norm"].corr(df_lh["behavior_label"], method="pearson")
        spear_qam = df_lh["QAM_norm"].corr(df_lh["behavior_label"], method="spearman")

        pear_sam  = df_lh["SAM_norm"].corr(df_lh["behavior_label"], method="pearson")
        spear_sam = df_lh["SAM_norm"].corr(df_lh["behavior_label"], method="spearman")

        rows.append({
            "dataset": dataset,
            "layer": layer,
            "head": head,
            "pearson_PAM": pear_pam,
            "pearson_QAM": pear_qam,
            "pearson_SAM": pear_sam,
            "spearman_PAM": spear_pam,
            "spearman_QAM": spear_qam,
            "spearman_SAM": spear_sam,
        })

corr_df = pd.DataFrame(rows)
corr_df.head()


,dataset,layer,head,pearson_PAM,pearson_QAM,pearson_SAM,spearman_PAM,spearman_QAM,spearman_SAM
0,alpaca,0,0,0.234101,-0.226932,0.057725,0.270666,-0.257133,-0.067666
1,alpaca,0,1,0.274594,0.085872,-0.289340,0.270666,0.094733,-0.297733
2,alpaca,0,10,0.277086,-0.270821,-0.000533,0.324799,-0.324799,0.013533
3,alpaca,0,11,0.229500,-0.205120,-0.051367,0.284199,-0.297733,-0.027067
4,alpaca,0,12,0.255312,-0.242553,0.003999,0.324799,-0.270666,-0.027067


In [18]:
corr_df.to_csv("correlation_results.csv", index=False)
print("Saved correlation_results.csv")


Saved correlation_results.csv


In [19]:
import os

FIG_DIR = "/content/figures_week3/"
os.makedirs(FIG_DIR, exist_ok=True)
FIG_DIR


'/content/figures_week3/'

In [20]:
import seaborn as sns
import matplotlib.pyplot as plt

def make_scatter(df, metric):
    plt.figure(figsize=(6,4))
    sns.regplot(
        data=df,
        x=metric,
        y="behavior_label",
        scatter_kws={"s": 5, "alpha": 0.3}
    )
    plt.title(f"{metric} vs Behavior Label")
    out = f"{FIG_DIR}{metric}_scatter.png"
    plt.savefig(out, dpi=300, bbox_inches="tight")
    plt.close()
    print("Saved:", out)


In [21]:
make_scatter(merged, "PAM_norm")
make_scatter(merged, "QAM_norm")
make_scatter(merged, "SAM_norm")


Saved: /content/figures_week3/PAM_norm_scatter.png
Saved: /content/figures_week3/QAM_norm_scatter.png
Saved: /content/figures_week3/SAM_norm_scatter.png


In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

heat_pam = corr_df.pivot_table(
    index="layer",
    columns="dataset",
    values="pearson_PAM"
)

plt.figure(figsize=(8,6))
sns.heatmap(heat_pam, cmap="coolwarm", center=0)
plt.title("PAM Pearson Correlation by Layer")
plt.savefig(FIG_DIR + "PAM_layer_heatmap.png", dpi=300, bbox_inches="tight")
plt.close()

print("Saved PAM_layer_heatmap.png")


Saved PAM_layer_heatmap.png


In [23]:
behavior_summary = merged.groupby(
    ["dataset", "behavior_label"]
)[["PAM_norm", "QAM_norm", "SAM_norm"]].mean()

behavior_summary


PAM_norm  QAM_norm  SAM_norm
dataset  behavior_label                              
alpaca   0.0             0.820084  0.137648  0.042269
         1.0             0.839892  0.120712  0.039396
flan     0.0             0.929079  0.037429  0.033492
         1.0             0.927031  0.038193  0.034776
sharegpt 0.0             0.588939  0.078904  0.332157
         1.0             0.585542  0.081108  0.333350

In [24]:
behavior_summary.to_csv("/content/behavior_means.csv")
print("Saved behavior_means.csv")


Saved behavior_means.csv




# **Summary of Correlation Visualizations**

### **1. PAM Pearson Layer Heatmap**

This heatmap shows how **Prompt Attention (PAM_norm)** correlates with correct behavior across layers of the model, separately for each dataset (alpaca, flan, sharegpt).

**Key observations:**

* **Alpaca** shows the strongest positive correlations, especially in early layers (0–6). This suggests Alpaca examples rely more on **system-prompt compliance**.
* **Flan** correlations hover near zero or slightly negative, indicating the model does *not* depend strongly on the system prompt to behave correctly.
* **ShareGPT** sits in the middle: mild positive correlations in several middle layers but nothing extreme.
* Across all datasets, deeper layers (20–31) converge around zero, meaning **later layers rely less on prompt information** for determining behavior.

**Interpretation:**
Early layers encode prompt-following tendencies, especially in Alpaca. This lines up with intuition: Alpaca’s instructions are rigid and short, so the model leans on system prompts more.

---

# **2. PAM_norm vs Behavior Label (Scatter + Regression)**

**What we plotted:**
Each point = one model generation, colored by whether it was labeled as "correct" (1) or "incorrect" (0). The regression line shows the trend.

**Findings:**

* The regression slope is slightly **negative**.
* This means **higher PAM_norm very weakly predicts worse behavior**.
* But the effect is extremely small — most points cluster at behavior_label = 0 or 1.

**Interpretation:**
Prompt attention *alone* is not a strong predictor of correctness. Excessive focus on the system prompt might even be a mild distractor in certain datasets.

---

# **3. QAM_norm vs Behavior Label**

This plot shows a very different pattern.

**Findings:**

* The regression slope is clearly **positive**.
* Higher **User Attention (QAM_norm)** strongly associates with correct behavior.
* Points are more spread out → QAM_norm actually varies meaningfully across samples (unlike PAM_norm which is often near 0 or 1).

**Interpretation:**
This strongly supports the hypothesis:

> **Models behave correctly when they pay attention to the user’s actual question.**

This is the strongest, cleanest signal in all our plots.

---

# **4. SAM_norm vs Behavior Label**

**Findings:**

* The regression slope is slightly **negative**.
* Higher **Self-Attention (SAM_norm)** tends to correlate with *worse* behavior.
* Many incorrect samples show the model focusing on its own previous output rather than the prompt or the user.

**Interpretation:**
When the model attends too heavily to its own prior generations, it becomes more likely to:

* drift,
* hallucinate,
* or ignore instructions.

This supports the idea that **self-reinforcement leads to behavioral failure**.

---

# **Overall Takeaways Across All Plots**

### **1. QAM_norm (User Attention)**

The strongest predictor of correctness.
More attention to the user → better alignment.

### **2. PAM_norm (Prompt Attention)**

Weak or slightly negative correlation.
System prompt is useful but not the main driver of correct behavior.

### **3. SAM_norm (Self Attention)**

Consistently negative correlation.
Model drifting into its own previous outputs harms alignment.

### **4. Dataset Differences**

* Alpaca relies heavily on prompt attention (high PAM correlations).
* Flan relies on user attention more (higher QAM correlations).
* ShareGPT sits in the middle.

---



